<a href="https://colab.research.google.com/github/ImPeekaboo/mytools/blob/main/Cloud_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Setup:**

In [ ]:
#@markdown <br><center><img src="https://cdn.thecaduceus.eu.org/MCT/img/Setup_img.png" height="50" alt="Setup-Logo"/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from time import sleep
from requests import get as get_url
from urllib.parse import urlparse as validate_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
    json_data = get_url("https://cdn.thecaduceus.eu.org/MCT/json/mct.json").json()
    print("Connected with CDN.")
    if current_version == json_data["version"]:
        print(f'Great Job! you are using latest version: {json_data["version"]}')
    else:
        print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]}, use latest notebook available here:\n{json_data["notebookLink"]}')
        sleep(10)
except Exception as error_code:
    end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# Flavor Flags  
Stable = ""
Beta = "-s beta"

# User Input
Install_Rclone_Flavor = Stable #@param ["Stable", "Beta"] {type:"raw"}
Force_Setup = False #@param {type:"boolean"}
Custom_sh_file = False #@param {type:"boolean"}

if Custom_sh_file:

    Custom_sh_Source = input("Provide the Custom Install.sh file source, it can be URL or File Path:\n")
    use_sudo = input("Do you want to execute the .sh file as sudo? 1 = Yes and 0 = No:\n")

    if use_sudo not in ['0','1']:
        end("Unable to check for latest version!")
    elif not Custom_sh_Source:
        end("Custom sh file source can't be blank!")
    elif all([validate_url(Custom_sh_Source).scheme, validate_url(Custom_sh_Source).netloc]):
        try:
            get_url(Custom_sh_Source)
            if use_sudo == '1':
                !curl {Custom_sh_Source} | sudo bash
                print('Execution completed as sudo!')
            elif use_sudo == '0':
                !curl {Custom_sh_Source} | bash
                print('Execution completed!')
        except ConnectionError:
            end("Unable to connect with given sh file URL.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
    elif not Path(Custom_sh_Source).exists():
        end("Provided Custom sh file path not exist!")
    elif not Path(Custom_sh_Source).is_file():
        end("Provided Custom sh file path should of a particular file not directory/folder.")
    elif not PurePath(Custom_sh_Source).suffix == '.sh':
        end("File format should be .sh")
    else:
        try:
            if use_sudo == '1':
                !sudo bash "$Custom_sh_Source"
                print("Execution completed as sudo.") 
            else:
                !bash "$Custom_sh_Source"
                print("Execution complete.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")

    
    # Ask for Normal Setup
    do_normal_setup = input('Do you want to run Normal Setup also? 1 = Yes and 0 = No:\n')

    if do_normal_setup not in ['0','1']:
        end("Input Error: Value should be 1 or 0")

# Force Setup
if Force_Setup:
    if Path("/content/setup.sh").exists():
        !rm {"/content/setup.sh"}
    print("Starting Force Setup...")
    sleep(4)
    !curl "https://rclone.org/install.sh" | sudo bash {Install_Rclone_Flavor}

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
    try:
        get_url(json_data["setupFile"])
        !curl {json_data["setupFile"]} | sudo bash {Install_Rclone_Flavor}
        !curl {json_data["setupFile"]} > "/content/setup.sh"
        output.clear()
        print("Setup completed!")
    except ConnectionError:
        print('Unable to get setup file!')
    except Exception as error_code:
        print(f'Setup instructions seems to be having mistake(s)!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
    print("Setup file exist and configured already!")

In [ ]:
#@markdown <br><center><img src="https://cdn.thecaduceus.eu.org/MCT/img/Google-Drive-Logo.png" height="50" alt="Google-Drive-Logo"/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>

from pathlib import Path
from contextlib import suppress
from sys import exit as end
from google.colab import output
from google.colab.drive import mount, flush_and_unmount as unmount

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
FORCE_REMOUNT = True #@param {type:"boolean"}
CUSTOM_MOUNT_POINT = False #@param {type:"boolean"}
DEBUG_MODE = False #@param {type:"boolean"}

if CUSTOM_MOUNT_POINT:
  Custom_Mount_Path = input('Please enter Custom Mount Path:\n')
  if not Custom_Mount_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Mount_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Mount_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
else:
  Custom_Mount_Path = '/content/drive'

mount._DEBUG = DEBUG_MODE

if MODE == "MOUNT":
  print("Please Wait! Mounting...")
  mount(Custom_Mount_Path, force_remount= FORCE_REMOUNT)
  if not DEBUG_MODE:
    output.clear()
  print(f'Mounted at {Custom_Mount_Path}')
elif MODE == 'UNMOUNT':
  with suppress(ValueError):
    print('Please Wait! UNMOUNT in progress...')
    unmount()
  get_ipython().system_raw('rm -rf /root/.config/Google/DriveFS')

In [ ]:
#@markdown <br><center><img src="https://cdn.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center>
#@markdown <center><h3><b>Fetch Previous MCT Config from My Drive</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')

#@markdown ---
Custom_Import_Path = False #@param {type:"boolean"}
Custom_Export_Path = False #@param {type:"boolean"}

# Import Configuration(s)

if Custom_Import_Path:
  Custom_Path = input('Enter the directory path from where configuration file(s) needs to be imported:\n')
  # Check Path
  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
  elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder]')

  Custom_Path = PurePath(Custom_Path) # Remove any extra '/'

  # Paths
  IMPORT_TO = DRIVE_CONFIG_DIR
  IMPORT_RCLONE_CONF = f"{Custom_Path}/rclone.conf"
  IMPORT_ACCOUNTS_ZIP = f"{Custom_Path}/accounts.zip"
  IMPORT_ACCOUNTS_FOLDER = f"{Custom_Path}/accounts/"
  ZIP_ACCOUNTS_FOLDER = f"{CMP}/MyDrive/MCT-Config/accounts" # accounts is name of zip, zip file will be saved in MCT-Config
  IMPORT_MEGA_JSON = f"{Custom_Path}/mega.json"
  IMPORT_GOFILE_JSON = f"{Custom_Path}/gofile.json"

  # Check 'rclone.conf' file

  if Path(IMPORT_RCLONE_CONF).exists():

    is_import = input('Found: rclone.conf [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_RCLONE_CONF" "$IMPORT_TO"
      print('Successfully imported "rclone.conf" file! (1)')
    else:
      print('Not Found: rclone.conf [file]')

  # Check 'Service Accounts' file/folder

  if Path(IMPORT_ACCOUNTS_ZIP).exists() and Path(IMPORT_ACCOUNTS_FOLDER).exists():

    choice = input('Found both! "accounts.zip" [file] & "accounts" [folder]\n Select one: 0 = accounts.zip [file] or 1 = accounts [folder] or 2 = Ignore\n')

    if choice not in ['0','1','2']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif choice == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
    elif choice == '0':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_ZIP).exists():

    is_import = input('Found: accounts.zip [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_FOLDER).exists():

    is_import = input('Found: accounts [folder] | Do you want to import ? 0=No & 1=Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
  else:
    print('Not Found: accounts.zip [file] or accounts [folder]')

  # Check 'mega.json' file

  if Path(IMPORT_MEGA_JSON).exists():

    is_import = input('Found: mega.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_MEGA_JSON" "$IMPORT_TO"
      print('Successfully imported "mega.json" file! (1)')
        
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(IMPORT_GOFILE_JSON).exists():

    is_import = input('Found: gofile.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_GOFILE_JSON" "$IMPORT_TO"
      print('Successfully imported "gofile.json" file! (1)')
        
  else:
    print('Not Found: gofile.json [file]')

# Restore Configuration(s) from MCT-Config folder

if not Path(DRIVE_CONFIG_DIR).exists():
  end('Not Found: MCT-Config [folder]')
else:
  print('Found: MCT-Config [folder]')

# Paths
ROOT = "/content/"
DRIVE_RCLONE_CONF = f"{CMP}/MyDrive/MCT-Config/rclone.conf"
ROOT_RCLONE_CONF = "/root/.config/rclone/"
DRIVE_ACCOUNTS_ZIP = f"{CMP}/MyDrive/MCT-Config/accounts.zip"
DRIVE_MEGA_JSON = f"{CMP}/MyDrive/MCT-Config/mega.json"
DRIVE_GOFILE_JSON = f"{CMP}/MyDrive/MCT-Config/gofile.json"

# Check 'rclone.conf' file

if Path(DRIVE_RCLONE_CONF).exists():
  !rclone copy "$DRIVE_RCLONE_CONF" "$ROOT_RCLONE_CONF"
  print('Restored: rclone.conf [file]')
else:
  print('Not Found: rclone.conf [file]')

# Check 'Service Accounts' file

if Path(DRIVE_ACCOUNTS_ZIP).exists():
  !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$ROOT"
  print('Restored: accounts.zip [file]')
else:
  print('Not Found: accounts.zip [file]')

# Check 'mega.json' file

if Path(DRIVE_MEGA_JSON).exists():
  !rclone copy "$DRIVE_MEGA_JSON" "$ROOT"
  print('Restored: mega.json [file]')
else:
  print('Not Found: mega.json [file]')

# Check 'gofile.json' file

if Path(DRIVE_GOFILE_JSON).exists():
  !rclone copy "$DRIVE_GOFILE_JSON" "$ROOT"
  print('Restored: gofile.json [file]')
else:
  print('Not Found: gofile.json [file]')

if Custom_Export_Path:

  #Paths
  EXPORT_TO = f"{Custom_Path}/MCT-Config/"

  Custom_Path = input('Enter the directory path to where configuration file(s) needs to be exported:\n')

  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Path not found!")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Path is not a valid directory.")
  elif not Path(DRIVE_CONFIG_DIR):
    !rclone mkdir "$DRIVE_CONFIG_DIR"

  # Check 'rclone.conf' file

  if Path(DRIVE_RCLONE_CONF).exists():

    is_export = input('Found: rclone.conf [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_RCLONE_CONF" "$EXPORT_TO"
      print('Export: rclone.conf [file] (1)')
  else:
    print('Not Found: rclone.conf [file]')

    # Check 'Service Accounts' file

  if Path(DRIVE_ACCOUNTS_ZIP).exists():

    is_export = input('Found accounts.zip [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$EXPORT_TO"
      print('Export: accounts.zip [file] (1)')
  else:
    print('Not Found: accounts.zip [file]')

  # Check 'mega.json' file

  if Path(DRIVE_MEGA_JSON).exists():

    is_export = input('Found mega.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_MEGA_JSON" "$EXPORT_TO"
      print('Export: mega.json [file] (1)')
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(DRIVE_GOFILE_JSON).exists():

    is_export = input('Found gofile.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0', '1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_GOFILE_JSON" "$EXPORT_TO"
      print('Export: gofile.json [file] (1)')
  else:
    print('Not Found: gofile.json [file]')

####**Download Tools:**

In [ ]:
from libtorrent import session as lt_session, add_magnet_uri, torrent_info
from sys import exit as end, stdout
from httpx import post as post_url, get as get_url
from pathlib import Path, PurePath
from google.colab import files as colab_file
from os import makedirs, strerror
from json import load as json_load, dumps as dump_json
from subprocess import Popen, PIPE, STDOUT, run
from contextlib import closing
from functools import wraps
from errno import ETIME
from signal import signal, alarm, SIGALRM
from getpass import getpass
from time import time, sleep
import shlex

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder] -> MyDrive')

# Retrive Json Data
if Path('mct.json').exists():
  with open('mct.json','r') as mct_json:
    json_data = json_load(mct_json)
else:
  print("[INFO] - Downloading mct.json [file]")
  json_data = get_url("https://cdn.thecaduceus.eu.org/MCT/json/mct.json").json()
  with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# User Input
Source = ""  # @param {type:"string"}
Mode = "Direct Download Links" # @param ["Direct Download Links", "Download Links from txt file", "YouTube Videos/Playlist & Other Sites", "Torrent DL", "Torrent DL (File Selection)", "Upload file to Anonymous Platforms","MEGA Link DL", "UUP DUMP"]
Destination = "/content/drives/Onedrive"  # @param {type:"string"}
args = ""  # @param {type:"string"}


def runSh(args, *, output=False, shell=False, cd=None):
  if shell:
    return ((run(
        args,
        shell=True,
        stdout=PIPE,
        stderr=STDOUT,
        cwd=cd,
    ).stdout.decode("utf-8").strip()) if output else run(
        args, shell=True, cwd=cd).returncode)
  if output:
    proc = Popen(shlex.split(args), stdout=PIPE, stderr=STDOUT, cwd=cd)
    while True:
      output = proc.stdout.readline()
      if output == b"" and proc.poll() is not None:
        return
      if output:
        print(output.decode("utf-8").strip())
  return run(shlex.split(args), cwd=cd).returncode

def mega_dl():
  global Destination
  if not Destination:
    makedirs("downloads", exist_ok=True)
    Destination = "downloads"
  newlines = ['\n', '\r\n', '\r']
  def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with closing(stream):
      while True:
        out = []
        last = stream.read(1)
        if last == '' and proc.poll() is not None:
          break
        while last not in newlines:
          if last == '' and proc.poll() is not None:
            break
          out.append(last)
          last = stream.read(1)
        out = ''.join(out)
        yield out

  def transfer():
    cmd = ["mega-get", Source, f"{Destination}"]
    proc = Popen(
      cmd,
      stdout=PIPE,
      stderr=STDOUT,
      universal_newlines=True,
    )
    for line in unbuffered(proc):
      print(line)
  transfer()

def config_creds():
  class TimeoutError(Exception):
    pass

  def timeout(seconds=10, error_message=strerror(ETIME)):
    def decorator(func):
      def _handle_timeout(signum, frame):
        raise TimeoutError(error_message)

      def wrapper(*args, **kwargs):
        signal(SIGALRM, _handle_timeout)
        alarm(seconds)
        try:
          result = func(*args, **kwargs)
        finally:
          alarm(0)
        return result
      return wraps(func)(wrapper)
    return decorator

  @timeout(10)
  def runShT(args):
    return runSh(args, output=True)
  if not Path("/usr/bin/mega-cmd").exists():
    print("[ERROR] - MEGA not installed.")
  else:
    !pkill mega-cmd

  with open('mega.json', 'r') as mega_json:
    mega_json = json_load(mega_json)
    print('[INFO] - Decrypting MEGA credentials.')
  try:
    decrypted_mega_id = post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": f"{mega_json['mega_id']}"}).json()
    decrypted_mega_pass = post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": f"{mega_json['mega_pass']}"}).json()
  except Exception as error_code:
    end(f"Unable to decrypt MEGA Credentials.\nTraceback:\n{error_code}")

  try:
    runShT(f"mega-login {decrypted_mega_id['result']} {decrypted_mega_pass['result']}")
  except TimeoutError:
    runSh('mega-whoami', output=True)
  mega_dl()

def save_creds():
  mega_id = input('Enter MEGA Email ID or Username:\n')
  mega_pass = getpass('Enter MEGA Account Password: (Input is hidden)\n')
  if not mega_id:
    end("Input Error: MEGA ID can't be blank!")
  elif not mega_pass:
    end("Input Error: MEGA Password can't be blank!")
  elif len(mega_pass) < 8:
    end("Input Error: MEGA Password must be of 8 digit or greater.")

  print('[INFO] - Encrypting MEGA Credentials')
  try:
    encrypted_mega_id = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": f"{mega_id}"}).json()
    encrypted_mega_pass = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": f"{mega_pass}"}).json()
  except Exception as error_code:
    end(f"Unable to encrypt MEGA Credentials.\nTraceback:\n{error_code}")
  encrypted_mega_json = {"mega_id": f"{encrypted_mega_id['result']}", "mega_pass": f"{encrypted_mega_pass['result']}"}
  with open('mega.json','w') as save_json:
    save_json.write(dump_json(encrypted_mega_json))
    print('Created: mega.json [file]')
    save_json.close()
  !rclone copy "/content/mega.json" "$DRIVE_CONFIG_DIR"
  print('Saved: mega.json -> MyDrive')
  config_creds()

if Mode =="Direct Download Links":
  links = Source.split(',')
  for link in links:
    %cd "$Destination"
    !aria2c "$link" $args
  print('Execution completed!')
elif Mode == "Download Links from txt file":
  %cd "$Destination"
  !aria2c -i "$Source" $args
  print('Execution completed!')
elif Mode == "YouTube Videos/Playlist & Other Sites":
  %cd "$Destination"
  !yt-dlp "$Source" $args
  print('Execution completed!')
elif Mode =="Torrent DL":
  ses = lt_session()
  ses.listen_on(6881, 6891)
  user_input = input("Enter magnet link or torrent file path:\n") if not Source else Source
  if user_input.startswith("magnet:"):
    params = {"save_path": Destination}
    handle = add_magnet_uri(ses, user_input, params)
  elif Path(user_input).exists():
    info = torrent_info(user_input)
    params = {"ti": info, "save_path": Destination}
    handle = ses.add_torrent(params)
  else:
    end("Invalid input or given path not exist.")
  
  while not handle.status().is_seeding:
    s = handle.status()
    print(f"\r{s.progress * 100:.2f}% | DL: {s.download_rate / 1000000:.2f} MB/s | UP: {s.upload_rate / 1000000:.2f} MB/s | Peers: {s.num_peers} | Seeds: {s.num_seeds}", end=" ")
    stdout.flush()
    sleep(1)

elif Mode == "Torrent DL (File Selection)":
  ses = lt_session()
  ses.listen_on(6881, 6891)
  user_input = input("Enter magnet link or torrent file path:\n") if not Source else Source
  if user_input.startswith("magnet:"):
    params = {"save_path": Destination}
    handle = add_magnet_uri(ses, user_input, params)
  elif Path(user_input).exists():
    info = torrent_info(user_input)
    params = {"ti": info, "save_path": Destination}
    handle = ses.add_torrent(params)
  else:
    end("Invalid input or given path not exist.")

  print("Downloading metadata...")
  start_time = time()
  while not handle.has_metadata():
    elapsed_time = time() - start_time
    if elapsed_time > 30: # Increase timeout time from here, by default its 30 seconds
      end("Torrent timeout, failed to load metadata.")

  print("Metadata downloaded.")

  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  print("Files in torrent:")
  for i, f in enumerate(files):
    print(f"{i+1}: {f.path} ({f.size / 1000000:.2f} MB)")
  
  file_nums = input("Enter file numbers to download [separated by space or dash (for range)]:").split()

  priorities = [0] * len(files)
  for n in file_nums:
    if "-" in n:
      start_stop = n.split("-")
      start = int(start_stop[0]) - 1
      stop = int(start_stop[1])
      if start_stop[0] >= start_stop[1]:
        end("Incorrect range.")
      for i in range(start, stop):
        priorities[i] = 1
    else:
      priorities[int(n) - 1] = 1

  handle.prioritize_files(priorities)

  print("Downloading...")
  while not handle.is_seed():
    s = handle.status()
    print(f"\r{s.progress * 100:.2f}% | DL: {s.download_rate / 1000000:.2f} MB/s | UP: {s.upload_rate / 1000000:.2f} MB/s | Peers: {s.num_peers} | Seeds: {s.num_seeds}", end=" ")
    stdout.flush()
    sleep(1)
elif Mode == "Upload file to Anonymous Platforms":
  if not Source:
    end("Source path can't be empty!")
  elif not Path(Source).exists():
    end("Not Found: Given path not exists!")

  API = list(json_data['apis']['anonFileShare'].values())
  site_names = ""

  for num, name in enumerate(list(json_data['apis']['anonFileShare'].keys())):
    site_names += f"{num}.{name}\n"

  while True:
    count = int(input(f"Choose Platform by its number (0 - {len(API) - 1}):\n{site_names}>"))

    if count > len(API) - 1:
      print('Selection out of list.')
    else:
      print(f'Selection -> {count}\nAPI -> {API[count]}\n')
      break

  if not Path(Source).exists():
    end('Not Found: Given source path not exist.')
  elif Path(Source).is_dir():
    links = []
    is_recursive = input("Upload files recursively? 0 = No & 1 = Yes:\n")
    if is_recursive == '1':
      print("Scanning path for all possible directories...")
      for file_path in list(Path(Source).rglob('**/*.*')):
      # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
        if Path(file_path).is_file():
          try:
            print(f"Uploading [{PurePath(file_path).name}]")
            api_resp = post_url(API[count], files = {'file': open(file_path,'rb')}).json()
            print(f"{PurePath(file_path).name} -> " + api_resp["data"]["file"]["url"]["full"])
            links.append(f"{PurePath(file_path).name}:\n" + api_resp["data"]["file"]["url"]["full"])
          except OverflowError:
            print(f"File [{PurePath(file_path).name}] not allowed.")
          except KeyError as error_code:
            print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
          except Exception as error_code:
            print(f"An error occured!\nTraceback:\n{error_code}")
            sleep(4)
        else:
          print(f"[Dir] -> {PurePath(file_path).name}")
    else:
      for file_path in list(Path(Source).rglob("*.*")):
        if Path(file_path).is_file():
          try:
            print(f"Uploading [{PurePath(file_path).name}]")
            api_resp = post_url(API[count], files = {'file': open(file_path,'rb')}).json()
            print(f"{PurePath(file_path).name} -> " + api_resp["data"]["file"]["url"]["full"])
            links.append(f"{PurePath(file_path).name}:\n" + api_resp["data"]["file"]["url"]["full"])
          except OverflowError:
            print(f"File [{PurePath(file_path).name}] not allowed.")
          except KeyError as error_code:
            print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
          except Exception as error_code:
            print(f"An error occured!\nTraceback:\n{error_code}")
            sleep(4)
        else:
          print(f"[Dir] -> {PurePath(file_path).name}")
                            
    if len(links) > 1:
      print(f"Total Links -> {len(links)}")
      is_export = input("Do you want to export all links in txt file? 0 = No or 1 = Yes:\n")
      if is_export not in ['0','1'] or is_export == '1':
        with open('links.txt','a+') as save_links:
          save_links.writelines('%s\n' % item for item in links)
          save_links.close()
          colab_file.download('links.txt')
          print('Request: Initiated download -> links.txt [file]')
      else:
        with open('links.txt','w+') as save_links:
          save_links.ritelines('%s\n' % item for item in links)
          save_links.close()
          print("Saved: links.txt [file] -> Runtime")
    else:
      print(f'Total Links -> {len(links)}')
  elif Path(Source).is_file():
    try:
      print(f"Uploading [{PurePath(Source).name}]")
      api_resp = post_url(API[count], files={'file': open(Source, 'rb')}).json()
      print(f"{PurePath(Source).name} -> " + api_resp["data"]["file"]["url"]["full"])
    except OverflowError:
      print("File not allowed.")
    except Exception as error_code:
      print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")

elif Mode =="MEGA Link DL":
  use_account = input('Do you want to use your MEGA Account? to lift limits imposed on anonymous downloader? 0 = No & 1 = Yes:\n')
  if use_account not in ['0','1']:
    end("Input Error: Invalid input parameter! Please enter 0 or 1 as input.")
  elif use_account == '1' and Path('/content/mega.json').exists():
    use_existing = input('[INFO] - Found existing MEGA Account credentials, do you want to use them? 0 = No & 1 = Yes:\n')
    if use_existing not in ['0','1']:
      end("Input Error: Invalid input parameter! Please enter 0 or 1 as input.")
    elif use_existing == '1':
      config_creds()
    elif use_existing == '0':
      save_creds()
  elif use_account == '1':
    save_creds()
  else:
    mega_dl()

elif Mode == "UUP DUMP":
  if Path(Source).is_file() and PurePath(Source).name() == 'uup_download_linux.sh':
    !bash "$Source"
  elif Path(Source).is_dir() and Path(f"{Source}/uup_download_linux.sh").exists():
    !bash "$Source/uup_download_linux.sh"
  else:
    print("Not Found: uup_download_linux.sh [file]")

####**Packing & Extraction:**

In [ ]:
from sys import exit as end
from pathlib import Path, PurePath
from getpass import getpass

# User Input
Action = "UnZip" # @param ["Zip", "Zip with Password", "RaR", "RaR with Password", "7z", "7z with Password", "Tar", "Tar.gz", "Tar.bz2", "UnZip", "UnZip with Password", "UnRaR", "UnRaR with Password", "Extract 7z", "Extract 7z with Password", "Extract Tar", "Extract Tar.gz", "Extract Tar.bz2"]
Source_Path = "" #@param {type:"string"}
Packed_File_Name = "" #@param {type:"string"}
Destination_Path = "/content/drives/Onedrive" #@param {type:"string"}
args = "" #@param {type:"string"}
ENCRYPT_FILENAME = False  # @param {type:"boolean"}

# Check for Setup & common things
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Action:
  end("Select a supported operation to perform.")
elif not Source_Path:
  end("Source field can't be empty.")
elif not Destination_Path:
  end("Destination field can't be empty")
elif not Path(Source_Path).exists():
  end("Not found: Given source path not exist.")
elif not Path(Destination_Path).exists():
  end("Not found: Given destination path not exist.")
else:
  Source_Path = PurePath(Source_Path)
  Destination_Path = PurePath(Destination_Path)
  Packed_File_Name = PurePath(Source_Path).name if not Packed_File_Name else Packed_File_Name

if ENCRYPT_FILENAME and not Action.endswith("with Password") or Action == "Zip with Password":
  print("[WARN] - Selected action not support ENCRYPT_FILENAME, continuing without it.")

if Action == "Zip":
  Pack_Dest = f"{Destination_Path}/{Packed_File_Name}.zip"
  !zip -r "$Pack_Dest" "$Source_Path" $args
elif Action == "Zip with Password":
  Pack_Dest = f"{Destination_Path}/{Packed_File_Name}.zip"
  password = getpass("Enter password:(Input is hidden)\n")
  !zip -P "$password" -r "$Pack_Dest" "$Source_Path" $args
elif Action == "UnZip":
  !unzip "$Source_Path" -d "$Destination_Path" $args
elif Action == "UnZip with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !unzip -P "$password" -d "$Destination_Path" "$Source_Path" $args
elif Action == "RaR":
  Rar_Dest = f"{Destination_Path}/{Packed_File_Name}.rar"
  !rar a "$Rar_Dest" "$Source_Path" $args
elif Action == "RaR with Password":
  Rar_Dest = f"{Destination_Path}/{Packed_File_Name}.rar"
  password = getpass("Enter password:(Input is hidden)\n")
  if ENCRYPT_FILENAME:
    !rar a -p"$password" -hp "$Rar_Dest" "$Source_Path" $args
  else:
    !rar a -p"$password" "$Rar_Dest" "$Source_Path" $args
elif Action == "UnRaR":
  !unrar x "$Source_Path" "$Destination_Path" $args
elif Action == "UnRaR with Password":
  !unrar x -Y "$Source_Path" "$Destination_Path" $args
elif Action == "7z":
  sz_Dest = f"{Destination_Path}/{Packed_File_Name}.7z"
  !7z a "$sz_Dest" "$Source_Path" $args
elif Action == "7z with Password":
  sz_Dest = f"{Destination_Path}/{Packed_File_Name}.7z"
  password = getpass("Enter password:(Input is hidden)\n")
  if ENCRYPT_FILENAME:
    !7z a -p"$password" -mhe=on "$sz_Dest" "$Source_Path" $args
  else:
    !7z a -p"$password" "$sz_Dest" "$Source_Path" $args
elif Action == "Extract 7z":
  !7z x "$Source_Path" -o"$Destination_Path" $args
elif Action == "Extract 7z with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !7z x -p"$password" -o"$Destination_Path" "$Source_Path" $args
elif Action == "Tar":
  Tar_Dest = f"{Destination_Path}/{Packed_File_Name}.tar"
  !tar -cvf "$Tar_Dest" "$Source_Path" $args
elif Action == "Extract Tar" :
  !tar -xvf "$Source_Path" -C "$Destination_Path" $args
elif Action == "Tar.gz":
  Targz_Dest = f"{Destination_Path}/{Packed_File_Name}.tar.gz"
  !tar -czvf "$Targz_Dest" "$Source_Path" $args
elif Action == "Extract Tar.gz":
  !tar -xzvf "$Source_Path" -C "$Destination_Path" $args
elif Action == "Tar.bz2":
  !tar -cjvf "$Tarbz_Dest" "$Source_Path" $args
elif Action == "Extract Tar.bz2":
  Tarbz_Dest = f"{Destination_Path}/{Packed_File_Name}.tar.bz"
  !tar -xjvf "$Source_Path" -C "$Destination_Path" $args
else:
  print("Invalid Operation.")

####**Execute Your Commands here:**

In [ ]:
# @markdown <br>
# @markdown <center><h3><b>Connect To Cloud Accounts</b></h3></center>
# @markdown <br><center><img src="https://cdn.thecaduceus.eu.org/MCT/img/Multiple-Clouds.webp" height="100" alt="Multiple-Clouds"/></center>
# @markdown <center><b>Connect your Cloud Accounts (Rclone remotes) between which you want to transfer Data!</b></center>

from sys import exit as end
from os import makedirs, system
from pathlib import Path, PurePath
from re import findall, M

if not Path("/root/.config/rclone/rclone.conf").exists():
  end('Not Found: rclone.conf [file]')

content = open("/root/.config/rclone/rclone.conf").read()
avCon = findall(r"^\[(.+)\]$", content, M)

print("Available Rclone remotes:")
print(*avCon)
remote = input("Enter Rclone remote name to continue:\n")

if remote not in avCon:
  end('Not Found: Given remote name not exist in rclone.conf [file]')

# User Input
Custom_Mount_Point = False  # @param {type:"boolean"}
Custom_Cache_Dir = False  # @param {type:"boolean"}

# Custom Mount Point
if Custom_Mount_Point:
  Mount_Point = input("Enter Custom Mount path:\n")
  if not Mount_Point:
    end("Mount Path can't be empty.")
  elif Path(Mount_Point).exists() and Path(Mount_Point).is_dir():
      Mount_Point = f"{PurePath(Mount_Point)}/{remote}"
  elif not Path(Mount_Point).exists():
      end('Not Found: Given mount path not found.')
  elif not Path(Mount_Point).is_dir():
    end('Path Error: Given path is not a valid directory.')
else:
  Mount_Point = f"/content/drives/{remote}"

# Custom Cache Directory
if Custom_Cache_Dir:
  Cache_dir = input("Enter Custom Cache Directory path:\n")
  if not Cache_dir:
    end("Cache Directory Path can't be empty.")
  elif Path(Cache_dir).exists() and Path(Cache_dir).is_dir():
    Cache_dir = f"{PurePath(Cache_dir)}/{remote}"
  elif not Path(Cache_dir).exists():
    end('Not Found: Given mount path not found.')
  elif not Path(Cache_dir).exists():
    end('Path Error: Give path is not a valid directory.')
else:
  Cache_dir = "/content/temp/Rclone"

Action = input("Select Action: 1 = Mount and 0 = Unmount:\n")

if Action not in ['0','1']:
  end("Input Error: Value should be 1 or 0.")
elif Action == '1':
  makedirs(Mount_Point, exist_ok=True)
  !rclone mount {remote}: {Mount_Point} --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir {Cache_dir} --allow-other --daemon
  print(f'Mounted: {Mount_Point}') if Path(Mount_Point).exists() else print(f"Mount Error: Unable to mount {Mount_Point}")
elif Action == '0' and system(f"fusermount -uz {Mount_Point}") == 0:
  !rm -r {Mount_Point}
else:
  system(f"fusermount -uz {Mount_Point}")